In [8]:
import numpy as np
import pandas as pd
from funcs.model import MLP

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


In [2]:
df = pd.read_excel("data/Dry_Bean_Dataset.xlsx")

In [3]:
y = df["Class"]
y = y.astype("category").cat.codes
X = df.iloc[:,:-1]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.2, random_state=42)

In [14]:
class BeansDataset(Dataset):
    
    def __init__(self, X:np.array, y:np.array, transform=None):
        """
        Args:
            X: Features.
            y: target.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.X = X.astype('float32')
        self.y = y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        self.n_samples = self.X.shape[0]
        self.transform = transform

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        
        sample = self.X[idx], self.y[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    
batch_size = 1500

test_dataset = BeansDataset(X_test.to_numpy(), y_test.to_numpy())
train_dataset = BeansDataset(X_train.to_numpy(), y_train.to_numpy())

# This will apply the softmax
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=True, batch_size=batch_size)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

In [21]:
model = MLP(16,10,7)
device="cpu"
num_epochs=5
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss() # specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    # enumerate mini batches
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        # clear the gradients
        optimizer.zero_grad()
        # compute the model output
        yhat = model(inputs)
        # calculate loss
        print(yhat.shape)
        print(targets)
        loss = criterion(yhat, targets)
        # credit assignment
        loss.backward()
        # update model weights
        optimizer.step()
        print(f"epoch {epoch + 1} / {num_epochs}, step {i + 1}/{n_total_steps}, loss = {loss.item():.4f}")

torch.Size([1500, 7])
tensor([[4.],
        [6.],
        [0.],
        ...,
        [3.],
        [6.],
        [6.]])


RuntimeError: 1D target tensor expected, multi-target not supported

In [ ]:
# number of epochs to train the model
n_epochs = 50# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinityfor epoch in range(n_epochs):
# monitor losses
train_loss = 0
valid_loss = 0

###################
# train the model #
###################
model.train() # prep model for training
for data,label in train_loader:
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output,label)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    # update running training loss
    train_loss += loss.item() * data.size(0)


 ######################    
# validate the model #
######################
model.eval()  # prep model for evaluation
for data,label in valid_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output,label)
    # update running validation loss 
    valid_loss = loss.item() * data.size(0)

# print training/validation statistics 
# calculate average loss over an epoch
train_loss = train_loss / len(train_loader.sampler)
valid_loss = valid_loss / len(valid_loader.sampler)

print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
    epoch+1, 
    train_loss,
    valid_loss
    ))

# save model if validation loss has decreased
if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
    valid_loss_min,
    valid_loss))
    torch.save(model.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

In [22]:
yhat[0]

tensor([1., 1., 0., 1., 1., 0., 0.], grad_fn=<SelectBackward>)